In [4]:
def split_sessions(data):
    sessions = []
    current_session = []
    focus_level = None  # Initialize to None, will be updated with each focus level encountered
    
    for item in data:
        if item['type'] == 'focus_level':
            if current_session:  # Only proceed if there are events in the current session
                sessions.append((current_session, item['data']['level']))
            current_session = []  # Reset for the next sessionp
        else:
            # Accumulate events for the current session
            current_session.append(item)
    
    # If the data ends without a focus_level item, use the last known focus level for remaining events
    if current_session and focus_level is not None:
        sessions.append((current_session, focus_level))
    
    # Split the sessions into x and y for direct mapping
    x, y = zip(*sessions) if sessions else ([], [])
    
    return list(x), list(y)


In [1]:
import boto3
import os

# Initialize a session using Amazon S3
s3 = boto3.resource('s3')

# Define the bucket name
bucket_name = 'desk-top-activity'

# Define the base path within the S3 bucket (if you want to start from a specific directory)
base_path = ''

# Target directory where files will be downloaded
target_dir = '../../focus_level/'  # Current directory. You can change this to a specific path.

# Ensure the target directory exists
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Create a bucket object
bucket = s3.Bucket(bucket_name)

# Iterate over all objects in the bucket starting from the base path
for obj in bucket.objects.filter(Prefix=base_path):
    # Build the full local path for the downloaded file
    local_file_path = os.path.join(target_dir, obj.key)
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
    
    # Download the file
    print(f'Downloading {obj.key} to {local_file_path}...')
    bucket.download_file(obj.key, local_file_path)

print('Download complete.')


Download complete.
